# Attention Rank Visualizer — per head or whole layer

This notebook is based on the paper [When Attention Collapses: How Degenerate Layers in LLMs Enable Smaller, Stronger Models](https://arxiv.org/html/2404.08634v3).

1. Build the attention matrix `A` for a chosen head from the `CausalSelfAttention` module.
2. Measure an **effective rank**: the smallest `k` whose top singular values explain **90%** of the matrix energy.
3. Measure **single‑column‑ness**: the fewest columns needed to cover **90%** of the squared entries of `A`.
4. Plot a heatmap of `A` and a per‑head rank profile.

**Assumptions**
- Uses `models.gpt2.attention.CausalSelfAttention`'s merged QKVO weights in `qkvo_w` of shape `(4, num_heads*head_dim, dim)`.
- Hidden states `X` of shape `(T, dim)` that enter this attention block (from the previous layer).

**Tip**: Start with short sequences (e.g., `T ≤ 128`) because SVD (Singular Value Decomposition) scales cubically with `T`.


In [1]:
import torch
import matplotlib.pyplot as plt

import models.gpt2.block

torch.set_printoptions(precision=4, sci_mode=True)


## Metrics: effective rank (90%) and single‑column mass (90%)
- **effective_rank90(A)**: the smallest `k` so that the top `k` singular values of `A` explain at least 90% of the squared entries.
- **columns90(A)**: the fewest columns needed so that their squared entries sum to at least 90% of the squared entries of `A`.

Both are computed per head.


### Plot helpers

In [2]:
def show_attention_heatmap(A: torch.Tensor, head: int=0, title: str=None):
    """A: (H, T, T) or (T, T)."""
    plt.figure()
    if A.dim() == 3:
        M = A[head].detach().cpu().numpy()
    else:
        M = A.detach().cpu().numpy()
    plt.imshow(M, aspect='auto')
    plt.colorbar()
    plt.xlabel('Key position j')
    plt.ylabel('Query position i')
    if title:
        plt.title(title)
    plt.show()

def plot_head_ranks(ranks, title: str='Effective rank (90%) per head'):
    plt.figure()
    xs = list(range(len(ranks)))
    plt.plot(xs, ranks, marker='o')
    plt.xlabel('Head index')
    plt.ylabel('Rank-90%')
    plt.title(title)
    plt.show()

def plot_head_masses(masses, title: str='Fewest columns for 90% mass per head'):
    plt.figure()
    xs = list(range(len(masses)))
    plt.plot(xs, masses, marker='o')
    plt.xlabel('Head index')
    plt.ylabel('#columns for 90% mass')
    plt.title(title)
    plt.show()


In [3]:
from gpt2 import GPT2Core
from models.gpt2.attention import CausalSelfAttention
from tools.metrics.attn_rank import attention_matrix_from_attn, per_head_metrics, average_per_head_over_sequences
from training.data_gen import DistributedDataGenerator
from tools.checkpoint import model_from_checkpoint

pt = "/Users/jonathanmiddleton/models/checkpoints/350m-instruct/20251013T1953-val1.600-step000850-run1-best.pt"
device = 'cpu'
seq_len = 100 # 100 in paper

data_loader = DistributedDataGenerator(
    "../../data/fineweb/fineweb_val_000000.bin",
    1 * seq_len,
    rank = 0,
    world_size=1,
    device=device,
)
# noinspection PyTypeChecker
model: GPT2Core = model_from_checkpoint(pt, device=device, map_location=device).eval()
p = next(data_loader)[0][None,:] # one sample
with torch.no_grad():
    model.prefill_batch(p, 256)

def build_attention_matrix(layer_id):
    with torch.no_grad():
        # noinspection PyTypeChecker
        block: models.gpt2.block.Block = model.blocks[layer_id]
        X: torch.Tensor = block.in_t.squeeze(0) # (T, dim)
        return attention_matrix_from_attn(block.attn, X)

def compute_and_visualize(layer_id: int):
    with torch.no_grad():
        A = build_attention_matrix(layer_id)
        for h in range(A.shape[0]):
            show_attention_heatmap(A, head=h, title=f'Attention matrix — head {h}')
        ranks, masses, max_rank = per_head_metrics(A, device=device)
        print('Per‑head effective ranks:', ranks)
        print('Per‑head columns@90%:', masses)
        print('MaxRank(layer) =', max_rank)
        plot_head_ranks(ranks)
        plot_head_masses(masses)

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output

_layer_dropdown = widgets.Dropdown(
    options=list(range(len(model.blocks))),
    value=0,
    description='Layer:',
)
_run_button = widgets.Button(description='Run', button_style='primary')
_out = widgets.Output()

display(widgets.HBox([_layer_dropdown, _run_button]), _out)

def _on_run_clicked(_):
    _run_button.disabled = True
    _run_button.description = "Running..."
    _run_button.button_style = 'info'
    try:
        with _out:
            clear_output(wait=True)
            compute_and_visualize(int(_layer_dropdown.value))
    finally:
        _run_button.disabled = False
        _run_button.description = "Run"
        _run_button.button_style = 'primary'

_run_button.on_click(_on_run_clicked)


Output()

## Batch over multiple sequences (to average per head)
- Mirroring the paper's setup: sample `N=100` sequences with `T=100`, then compute per‑head averages and finally `MaxRank(l)` as the maximum head rank per layer.
- To inspect a **single‑column** pattern directly, sort the columns of `A[h]` by their squared mass and see if the first one dominates.


In [5]:
def list_f(l: list) -> str:
    return "[" + ", ".join(f"{v:.2f}" for v in l) + "]"

def compute_batch(layer_id: int):
    with torch.no_grad():
        I = [inputs[None,:] for inputs, _ in (next(data_loader) for _ in range(seq_len))]
        Xs = [model.blocks[layer_id].in_t[0] for inputs in I for _ in model.prefill_batch(inputs, seq_len)]

        avgs = average_per_head_over_sequences(model.blocks[layer_id].attn, Xs, device=device)
        print("avg_ranks_per_head: ", list_f(avgs['avg_ranks_per_head']))
        print("avg_columns90_per_head: ", list_f(avgs['avg_columns90_per_head']))
        print(f"MaxRank_layer: {avgs['MaxRank_layer']:.2f}")

In [7]:
import ipywidgets as widgets

_layer_dropdown_batch = widgets.Dropdown(
    options=list(range(len(model.blocks))),
    value=0,
    description='Layer:',
)
_run_button_batch = widgets.Button(description='Run', button_style='primary')
_out_batch = widgets.Output()

def _on_run_clicked_batch(_):
    _run_button_batch.button_style = 'info'
    _run_button_batch.description = "Running..."
    _run_button_batch.disabled = True
    try:
        with _out_batch:
            clear_output(wait=True)
            compute_batch(int(_layer_dropdown_batch.value))
    finally:
        _run_button_batch.button_style = 'primary'
        _run_button_batch.description = "Run"
        _run_button_batch.disabled = False

try:
    _run_button_batch.on_click(_on_run_clicked_batch, remove=True)
except Exception:
    pass
_run_button_batch.on_click(_on_run_clicked_batch)

display(widgets.VBox([
    widgets.HBox([_layer_dropdown_batch, _run_button_batch]),
    _out_batch
]))